## clinical variable retrieval codebase.
each querey is referencing a sql querey linked in my github for ALL patients in the database, then generating a dataframe, then paring that dataframe down to only the patients/icustay_id in our cohort. 
* 5-16-19 heavily streamlined, can now change global variables at top of page which will correspond to all variables. added all code into functions and made a composite function to run each variable. 
* each variable is also deleted to reduce rolling memory usage


### overall workflow of this notebook:
1) extract vitals first

2) limit patients to those with appropriate vitals going forward. (final_pt_df_v)

3) extract all other clinical variables 
 * after extracting all patients first, filter to those patints in minimum vitals list
 * save and delete after extracting to reduce unnescessary memory load

_note_: the sql scripts are currently setup to drop a materialized view if exists and create a new materialized view with every run.
 one could setup this workflow so that these views are only created once by removing this part in each sql script, or simply querying the view name instead.

In [1]:
#import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import collections
import asyncio
import getpass
import re
from datetime import datetime as dt
import os,sys,re
from collections import Counter
import seaborn as sns
import random
from pathlib import Path

%matplotlib inline
plt.style.use('ggplot')

from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'width': 1024,
        'height': 768,
        'scroll': True,
})

%load_ext autotime

time: 0 ns (started: 2023-07-26 17:46:58 +08:00)


In [2]:
# note, all server information is stored in a config.py file that is present in the .gitignore
import config 
conn = psycopg2.connect(dbname=config.dbname, user=config.user, host=config.host, port=config.port,password=config.password)
cur=conn.cursor()

query_schema = 'SET search_path to ' + "mimiciv" + ';'


time: 78 ms (started: 2023-07-26 17:47:00 +08:00)


In [3]:
from parameters import repository_path
os.chdir(repository_path)
wd= os.getcwd()

time: 3.52 s (started: 2023-07-26 17:47:03 +08:00)


In [31]:
final_pt_df2

,stay_id,ssc_charttime,ssc_id,icu_admit,ab_id,ab_start,ab_end,ab_ssc_delta,t_0,t_end,...,t_end_consec,subject_id,dod,ab_course,org_list,spec_type_list,first_pos_else_neg_ssc,sc_result,final_bin,hadm_id
0,37510196,2131-01-10 16:36:00,635,2131-01-11 04:20:05,14800413,2131-01-11 08:00:00,2131-01-20 12:00:00,0 days 15:24:00,2131-01-11 08:00:00,2131-01-20 12:00:00,...,2131-01-20 12:00:00,10001884,2131-01-20,full,NaN,NaN,2131-01-10 16:36:00,negative,C_neg/A_full,26184834
1,33685454,2129-08-05 15:54:00,796,2129-08-04 12:45:00,14801473,2129-08-05 14:00:00,2129-08-10 13:00:00,-1 days +22:06:00,2129-08-05 14:00:00,2129-08-10 13:00:00,...,2129-08-16 09:00:00,10002155,2131-03-10,full,NaN,NaN,2129-08-05 15:54:00,negative,C_neg/A_full,23822395
2,31090461,2130-09-25 11:04:00,822,2130-09-24 00:50:00,14801575,2130-09-25 11:00:00,2130-09-26 11:00:00,-1 days +23:56:00,2130-09-25 11:00:00,2130-09-26 11:00:00,...,2130-09-26 11:00:00,10002155,2131-03-10,partial,NaN,NaN,2130-09-25 11:04:00,negative,C_neg/A_partial,28994087
3,32358465,2131-03-10 00:58:00,828,2131-03-09 21:33:00,14801420,2131-03-10 01:00:00,2131-03-10 14:00:00,0 days 00:02:00,2131-03-10 01:00:00,2131-03-10 14:00:00,...,2131-03-10 14:00:00,10002155,2131-03-10,full,NaN,NaN,2131-03-10 00:58:00,negative,C_neg/A_full,20345487
4,38875437,2156-04-19 16:28:00,893,2156-04-19 18:11:19,14802091,2156-04-20 12:00:00,2156-04-21 11:00:00,0 days 19:32:00,2156-04-20 12:00:00,2156-04-21 11:00:00,...,2156-04-21 17:00:00,10002428,NaN,partial,NaN,NaN,2156-04-19 16:28:00,negative,C_neg/A_partial,28662225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27215,35165301,2197-08-05 14:57:00,3228511,2197-08-04 00:02:00,14796608,2197-08-04 20:00:00,2197-08-12 11:00:00,-1 days +05:03:00,2197-08-04 20:00:00,2197-08-12 11:00:00,...,2197-08-12 11:00:00,19999287,2197-09-02,full,NaN,NaN,2197-08-05 14:57:00,negative,C_neg/A_full,20175828
27216,37364566,2162-08-15 09:33:00,3228521,2162-08-16 05:48:32,14796834,2162-08-16 08:00:00,2162-08-17 12:00:00,0 days 22:27:00,2162-08-16 08:00:00,2162-08-17 12:00:00,...,2162-08-17 12:00:00,19999297,2162-08-23,partial,STAPH AUREUS COAG +,BLOOD CULTURE,2162-08-15 09:33:00,positive,C_pos/A_partial,21439025
27217,36075953,2149-01-08 20:00:00,3228661,2149-01-08 18:12:00,14798102,2149-01-08 20:00:00,2149-01-14 08:00:00,0 days 00:00:00,2149-01-08 20:00:00,2149-01-14 08:00:00,...,2149-01-14 08:00:00,19999828,NaN,full,NaN,NaN,2149-01-08 20:00:00,negative,C_neg/A_full,25744818
27218,38978960,2164-09-12 13:39:00,3228673,2164-09-12 09:26:28,14798283,2164-09-12 20:00:00,2164-09-16 12:00:00,0 days 06:21:00,2164-09-12 20:00:00,2164-09-16 12:00:00,...,2164-09-16 12:00:00,19999840,2164-09-17,full,NaN,NaN,2164-09-12 13:39:00,negative,C_neg/A_full,21033226


time: 171 ms (started: 2023-07-26 17:02:21 +08:00)


In [4]:
#patients of interest from rotation_cohort_generation
from parameters import final_pt_df, date, repository_path

#patients of interest from rotation_cohort_generation
final_pt_df2 = final_pt_df
del(final_pt_df)

patients= list(final_pt_df2['subject_id'].unique())
hadm_id= list(final_pt_df2['hadm_id'].unique())
stay_id= list(final_pt_df2['stay_id'].unique())
stay_id= [int(x) for x in stay_id]

time: 15 ms (started: 2023-07-26 17:47:07 +08:00)


In [5]:
#### # ONLY run if already have a final_pt_df_v stored (which is ensuring pts have adequate vitals to be considered suspected of BI). 
### this is to get around having to reimport vitals if want to reextract a single variable again
from parameters import final_pt_df_v
final_pt_df2= final_pt_df_v
icustay_id_vitals= list(final_pt_df_v['stay_id'].unique())

time: 0 ns (started: 2023-07-26 17:47:12 +08:00)


In [5]:
final_pt_df2

,stay_id,ssc_charttime,ssc_id,icu_admit,ab_id,ab_start,ab_end,ab_ssc_delta,t_0,t_end,...,t_end_consec,subject_id,dod,ab_course,org_list,spec_type_list,first_pos_else_neg_ssc,sc_result,final_bin,hadm_id
0,37510196,2131-01-10 16:36:00,635,2131-01-11 04:20:05,14800413,2131-01-11 08:00:00,2131-01-20 12:00:00,0 days 15:24:00,2131-01-11 08:00:00,2131-01-20 12:00:00,...,2131-01-20 12:00:00,10001884,2131-01-20,full,NaN,NaN,2131-01-10 16:36:00,negative,C_neg/A_full,26184834
1,33685454,2129-08-05 15:54:00,796,2129-08-04 12:45:00,14801473,2129-08-05 14:00:00,2129-08-10 13:00:00,-1 days +22:06:00,2129-08-05 14:00:00,2129-08-10 13:00:00,...,2129-08-16 09:00:00,10002155,2131-03-10,full,NaN,NaN,2129-08-05 15:54:00,negative,C_neg/A_full,23822395
2,31090461,2130-09-25 11:04:00,822,2130-09-24 00:50:00,14801575,2130-09-25 11:00:00,2130-09-26 11:00:00,-1 days +23:56:00,2130-09-25 11:00:00,2130-09-26 11:00:00,...,2130-09-26 11:00:00,10002155,2131-03-10,partial,NaN,NaN,2130-09-25 11:04:00,negative,C_neg/A_partial,28994087
3,32358465,2131-03-10 00:58:00,828,2131-03-09 21:33:00,14801420,2131-03-10 01:00:00,2131-03-10 14:00:00,0 days 00:02:00,2131-03-10 01:00:00,2131-03-10 14:00:00,...,2131-03-10 14:00:00,10002155,2131-03-10,full,NaN,NaN,2131-03-10 00:58:00,negative,C_neg/A_full,20345487
4,38875437,2156-04-19 16:28:00,893,2156-04-19 18:11:19,14802091,2156-04-20 12:00:00,2156-04-21 11:00:00,0 days 19:32:00,2156-04-20 12:00:00,2156-04-21 11:00:00,...,2156-04-21 17:00:00,10002428,NaN,partial,NaN,NaN,2156-04-19 16:28:00,negative,C_neg/A_partial,28662225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27215,35165301,2197-08-05 14:57:00,3228511,2197-08-04 00:02:00,14796608,2197-08-04 20:00:00,2197-08-12 11:00:00,-1 days +05:03:00,2197-08-04 20:00:00,2197-08-12 11:00:00,...,2197-08-12 11:00:00,19999287,2197-09-02,full,NaN,NaN,2197-08-05 14:57:00,negative,C_neg/A_full,20175828
27216,37364566,2162-08-15 09:33:00,3228521,2162-08-16 05:48:32,14796834,2162-08-16 08:00:00,2162-08-17 12:00:00,0 days 22:27:00,2162-08-16 08:00:00,2162-08-17 12:00:00,...,2162-08-17 12:00:00,19999297,2162-08-23,partial,STAPH AUREUS COAG +,BLOOD CULTURE,2162-08-15 09:33:00,positive,C_pos/A_partial,21439025
27217,36075953,2149-01-08 20:00:00,3228661,2149-01-08 18:12:00,14798102,2149-01-08 20:00:00,2149-01-14 08:00:00,0 days 00:00:00,2149-01-08 20:00:00,2149-01-14 08:00:00,...,2149-01-14 08:00:00,19999828,NaN,full,NaN,NaN,2149-01-08 20:00:00,negative,C_neg/A_full,25744818
27218,38978960,2164-09-12 13:39:00,3228673,2164-09-12 09:26:28,14798283,2164-09-12 20:00:00,2164-09-16 12:00:00,0 days 06:21:00,2164-09-12 20:00:00,2164-09-16 12:00:00,...,2164-09-16 12:00:00,19999840,2164-09-17,full,NaN,NaN,2164-09-12 13:39:00,negative,C_neg/A_full,21033226


time: 62 ms (started: 2023-07-25 11:29:08 +08:00)


## Access MIMIC database and convert it to dataframe in Pandas

In [6]:
#input the sql_exe_show object and get dataframe for only patients in patient list out. 
def sql_exe_show(sql_sentence):
    cur.execute(sql_sentence)
    rows = cur.fetchall()
    col = []
    for i in range(len(cur.description)):
        col.append(cur.description[i][0])
    table = pd.DataFrame(rows,columns=col)
    return table

def sql_to_df_icu(sql_exe_show_obj):
    sql_exe_show_df= pd.DataFrame(data=sql_exe_show_obj)
    sql_exe_show_df=sql_exe_show_df[sql_exe_show_df['stay_id'].isin(stay_id)]
    return sql_exe_show_df

def sql_to_df_patients(sql_exe_show_obj):
    sql_exe_show_df= pd.DataFrame(data=sql_exe_show_obj)
    sql_exe_show_df=sql_exe_show_df[sql_exe_show_df['subject_id'].isin(patients)]
    return sql_exe_show_df

def sql_to_df_hadm(sql_exe_show_obj):
    sql_exe_show_df= pd.DataFrame(data=sql_exe_show_obj)
    sql_exe_show_df=sql_exe_show_df[sql_exe_show_df['hadm_id'].isin(hadm_id)]
    return sql_exe_show_df

def clinvar_fxn(var_name, path, subject_id_override=False):
    f= open(path, 'r')
    var = f.read()
    cur.execute('rollback')
    cur.execute(var)
    
    if subject_id_override==True:
        df= sql_to_df_patients(sql_exe_show('select * from %s;' %(var_name)))
    
    else:
        
        try:
            df= sql_to_df_icu(sql_exe_show('select * from %s;' %(var_name)))
        except KeyError or NameError:
            try:      
                df= sql_to_df_hadm(sql_exe_show('select * from %s;' %(var_name)))
            except KeyError or NameError:
                df= sql_to_df_patients(sql_exe_show('select * from %s;' %(var_name)))

    print(df.shape)
    return(df)


time: 0 ns (started: 2023-07-26 17:47:16 +08:00)


# extracting clinical data for our patients
* clinical data window= (t0+x)- t0+y
 * lower_window: x, set this to offset the t_0 for lower bound of the clinical time window
 * upper_window: y, set this to set the upper bound of the clinical time window.
* folder: folder name for data to be stored in
* date: date attached in file name of all files associated with this data
* time_col: the time column used to restrict data to the clinical data window.
* patient_df: the cohort dataframe used, default: final_pt_df2
* save_boolean: # do you want to save the files generated?
    * if yes, save_boolean=True
    * else save_boolean=False

In [7]:
from parameters import lower_window, upper_window, folder, date, time_col, time_var, patient_df, save_boolean

time: 0 ns (started: 2023-07-26 17:47:20 +08:00)


# important functions

In [8]:
def time_window_filter(df, final_pt_df2, timecol, upper_window, lower_window, time_var='t_0'):
    """
    function: take in any df and filter to only values between lower_window and upper_window. 
    values = the difference between timecol-timevar. 
        most commonly this is charttime- t_0 (or first in ICU ABrx meeting criteria)
    will add delta and t_0 to df as well which are used lateron.git
    
    df- dataframe of values extracted from MIMIC
    final_pt_df2- my cohort dataframe. included here because it has my time_var(t_0) and icustay_id/hadm_id
    timecol- column in df used to define a measurement time
    time_var- the base time that timecol will be compared against (ie hospital admission, or first ab date(t_0))
    upper_window- time_var+ t, where t is in days (though can be changed below)
    lower_window- time_var+ t, where t is in days (though can be changed below)
    
    """
    #global upper_window, lower_window 
    
    try:
        df= pd.merge(df, final_pt_df2[['stay_id',time_var]], left_on= 'stay_id', right_on = 'stay_id') #n=240317
        df['delta']= pd.to_datetime(df[timecol]) - pd.to_datetime(df[time_var])
        df_after_t0= df.loc[df.loc[:,'delta']>= pd.Timedelta(days=lower_window),:]
        df_after_t0= df_after_t0.loc[df_after_t0.loc[:,'delta']<= pd.Timedelta(days=upper_window),:]
        
    except KeyError or NameError:
        df= pd.merge(df, final_pt_df2[['hadm_id',time_var]], left_on= 'hadm_id', right_on = 'hadm_id') #n=240317
        df['delta']= pd.to_datetime(df[timecol]) - pd.to_datetime(df[time_var])
        df_after_t0= df.loc[df.loc[:,'delta']>= pd.Timedelta(days=lower_window),:]
        df_after_t0= df_after_t0.loc[df_after_t0.loc[:,'delta']<= pd.Timedelta(days=upper_window),:]
    return(df_after_t0)

time: 0 ns (started: 2023-07-26 17:47:21 +08:00)


In [9]:
##### this is a quick getaround 

time: 544 µs


# variable extraction below:

## Vital Sign

In [9]:
##-- This query pivots the vital signs for the first 24 hours of a patient's stay
##-- Vital signs include heart rate, blood pressure, respiration rate, and temperature
from parameters import repository_path
vitals_all_nosummary_df= clinvar_fxn(
    'vitals_all_nosummary',
    str(repository_path)+'/src/SQL/vitals_all_nosummary.sql'
    )

(22828079, 7)
time: 5min 18s (started: 2023-07-26 17:47:24 +08:00)


In [9]:
vitals_all_nosummary_df

,subject_id,hadm_id,stay_id,charttime,valueuom,vitalid,valuenum
748,19812766,21501762,35193405,2143-02-20 04:29:00,%,SpO2,99.0
749,19812766,21501762,35193405,2143-02-20 04:30:00,bpm,HeartRate,114.0
750,19812766,21501762,35193405,2143-02-20 04:30:00,insp/min,RespRate,24.0
751,19812766,21501762,35193405,2143-02-20 04:30:00,%,SpO2,99.0
752,19812766,21501762,35193405,2143-02-20 04:32:00,mmHg,SysBP,143.0
...,...,...,...,...,...,...,...
41437571,19810100,21542247,35834097,2141-12-17 21:00:00,bpm,HeartRate,120.0
41437572,19810100,21542247,35834097,2141-12-17 21:00:00,insp/min,RespRate,20.0
41437573,19810100,21542247,35834097,2141-12-17 21:00:00,%,SpO2,93.0
41437574,19810100,21542247,35834097,2141-12-17 21:01:00,mmHg,SysBP,128.0


time: 109 ms (started: 2023-07-26 08:55:12 +08:00)


In [10]:
fio2_df= clinvar_fxn(
    'fio2_chartevents',
    str(repository_path)+'/src/SQL/fio2_chartevents.sql' 
)

(501705, 5)
time: 1min 21s (started: 2023-07-26 17:52:42 +08:00)


In [11]:
fio2_df['valueuom']= '%'
fio2_df['vitalid']= 'fio2_chartevents'

time: 15 ms (started: 2023-07-26 17:54:04 +08:00)


In [14]:
fio2_df

,subject_id,hadm_id,stay_id,charttime,valuenum,valueuom,vitalid
1,10001884,26184834,37510196,2131-01-11 04:00:00,100.0,%,fio2_chartevents
2,10001884,26184834,37510196,2131-01-11 11:00:00,50.0,%,fio2_chartevents
3,10001884,26184834,37510196,2131-01-11 12:00:00,50.0,%,fio2_chartevents
4,10001884,26184834,37510196,2131-01-11 16:00:00,50.0,%,fio2_chartevents
5,10001884,26184834,37510196,2131-01-11 19:00:00,50.0,%,fio2_chartevents
...,...,...,...,...,...,...,...
736213,19999987,23865745,36195440,2145-11-03 11:30:00,40.0,%,fio2_chartevents
736214,19999987,23865745,36195440,2145-11-03 16:00:00,40.0,%,fio2_chartevents
736215,19999987,23865745,36195440,2145-11-03 20:54:00,60.0,%,fio2_chartevents
736216,19999987,23865745,36195440,2145-11-03 22:37:00,50.0,%,fio2_chartevents


time: 15 ms (started: 2023-07-25 11:36:57 +08:00)


In [12]:
vitals_all_nosummary_df= vitals_all_nosummary_df._append(fio2_df, sort=False)

time: 1.61 s (started: 2023-07-26 17:54:04 +08:00)


In [16]:
vitals_all_nosummary_df

,subject_id,hadm_id,stay_id,charttime,valueuom,vitalid,valuenum
654,10001884,26184834,37510196,2131-01-18 19:00:00,bpm,HeartRate,69.0
655,10001884,26184834,37510196,2131-01-18 19:00:00,insp/min,RespRate,17.0
656,10001884,26184834,37510196,2131-01-18 19:01:00,mmHg,SysBP,122.0
657,10001884,26184834,37510196,2131-01-18 19:01:00,mmHg,DiasBP,67.0
658,10001884,26184834,37510196,2131-01-18 19:01:00,mmHg,MeanArtPress,84.0
...,...,...,...,...,...,...,...
736213,19999987,23865745,36195440,2145-11-03 11:30:00,%,fio2_chartevents,40.0
736214,19999987,23865745,36195440,2145-11-03 16:00:00,%,fio2_chartevents,40.0
736215,19999987,23865745,36195440,2145-11-03 20:54:00,%,fio2_chartevents,60.0
736216,19999987,23865745,36195440,2145-11-03 22:37:00,%,fio2_chartevents,50.0


time: 15 ms (started: 2023-07-25 11:36:58 +08:00)


# filtering to patients with bare minimum vital numbers
previously we found that 3% or so of patients don't have baseline vitals counts. this is filtering the patients to only those who have this baseline value.

In [13]:
#using origional criteria to find pts who have at least 1 spo2 reading within 3 days of t_0

#The idea is that this should be the bare minimum amount of data for a patient, and without it, it's likely the physicians did not suspect an infection in these patients. 
##NOTE: this should not change when the clinical timewindow of analysis interest changes. 

vitals_filter = time_window_filter(vitals_all_nosummary_df,
                                   final_pt_df2,
                                   time_col,
                                   time_var='t_0',
                                   lower_window=0,
                                   upper_window=3 )

vitals_filter= vitals_filter.loc[
    vitals_filter['vitalid'].notnull(),:]

icustay_id_vitals = (vitals_filter.loc[
    vitals_filter.loc[:,'vitalid']=='SpO2','stay_id'
        ].unique())

subject_id_vitals=list(final_pt_df2.loc[final_pt_df2.loc[:,'stay_id'].isin(icustay_id_vitals),'subject_id'])
hadm_id_vitals= list(final_pt_df2.loc[final_pt_df2.loc[:,'stay_id'].isin(icustay_id_vitals),'hadm_id'])
icustay_id_vitals= list(final_pt_df2.loc[final_pt_df2.loc[:,'stay_id'].isin(icustay_id_vitals),'stay_id'])
del(vitals_filter)

##saving the patient database and reassigning patient set to the patient set with minimum vitals

final_pt_df2_v=final_pt_df2.loc[final_pt_df2.loc[:,'stay_id'].isin(icustay_id_vitals),:].copy()
if save_boolean==True:
    #pd.DataFrame(final_pt_df2_v).to_csv("/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_final_pt_df2_v.csv" %date) #final cohort database n=11493 subject_id’s (7/6/18)
    pd.DataFrame(final_pt_df2_v).to_csv((str(repository_path)+ '/data/{}_final_pt_df_v.csv'.format(date)), index=False)
    final_pt_df2=final_pt_df2_v.copy()
 

time: 17.7 s (started: 2023-07-26 17:54:06 +08:00)


In [19]:
final_pt_df2=final_pt_df2_v
final_pt_df2_v['final_bin'].value_counts()
###weird the 24 hr #s are:
# C_neg/A_partial    14583
# C_neg/A_full        7802
# C_pos/A_partial     2551
# C_pos/A_full        1535

final_bin
C_neg/A_partial    14583
C_neg/A_full        7802
C_pos/A_partial     2551
C_pos/A_full        1535
Name: count, dtype: int64

time: 16 ms (started: 2023-07-25 12:43:24 +08:00)


## filtering, subset, and composite functions to be used for the rest of the notebook

In [14]:
def df_subset(df):
    """
    redundancy check to ensure all df are filtered to cohort with minimum vitals 
    """
    try:
        df = df.loc[df.loc[:,'stay_id'].isin(icustay_id_vitals),:]
    except KeyError or NameError:
        try:
            df = df.loc[df.loc[:,'hadm_id'].isin(hadm_id_vitals),:]
        except KeyError or NameError:
            df = df.loc[df.loc[:,'subject_id'].isin(subject_id_vitals),:]
    return(df)

time: 16 ms (started: 2023-07-26 17:54:23 +08:00)


In [15]:
def filter_subset_save(df, savename=None, return_df=False, save=False, time_filter_override=False):
    
    """
    composite function, performs 1: time_window_filter() and 2:df_subset() to the input dataframe. this function links them together for simplifying code needed after each sql and formatting query. 
    return_df specifies if any output is spit out.
    save specifies if the file will be saved with teh savename.
    
    fxn was created on 5/16/19 and validated against the normal pipeline.
    """
    
    global date,folder,final_pt_df2, lower_window, upper_window, time_var, timecol, time_var

    if time_filter_override==False:
        time_filtered= time_window_filter(df, final_pt_df2, timecol=time_col ,time_var=time_var, lower_window=lower_window, upper_window=upper_window)
    else:
        time_filtered=df
        
    time_and_subseted= df_subset(time_filtered)
    
    if save==True:
        if folder != None:
             address=str(repository_path)+'/data/raw/%s/'%(folder)
        else:
            address = str(repository_path)+'/data/raw/'
        if not os.path.exists(address):
            print(address)
            os.makedirs(address)
            
        pd.DataFrame(time_and_subseted).to_csv(address+'/%s_%s.csv' %(date, savename))       
    else: pass
    
    if return_df==False:
        del(df, time_filtered, time_and_subseted)
    
    else:
        return(time_and_subseted)
    

time: 0 ns (started: 2023-07-26 17:54:23 +08:00)


In [22]:
filter_subset_save(vitals_all_nosummary_df, savename="vitals_all_nosummary", save=save_boolean, return_df=False)

c:\Users\csfhe\Documents\RA\mimiciii-antibiotics-opensource\notebooks/data/raw/24_hr_window/
time: 1min 38s (started: 2023-07-25 13:06:53 +08:00)


In [ ]:
del(vitals_all_nosummary_df)

time: 2.72 s (started: 2023-07-26 12:11:35 +08:00)


## elixhauser comobridities

In [15]:
elixhauser_nosummary_df= clinvar_fxn(
    'elixhauser_quan',
    str(repository_path)+'/src/SQL/elixhauser_quan.sql', 
    subject_id_override=True
)

(37743, 33)
time: 40.7 s (started: 2023-07-26 14:38:15 +08:00)


In [16]:
print(elixhauser_nosummary_df.head().to_markdown())
elixhauser_nosummary_df.head()

|    |     hadm_id |   subject_id |   stay_num |   congestive_heart_failure |   cardiac_arrhythmias |   valvular_disease |   pulmonary_circulation |   peripheral_vascular |   hypertension |   paralysis |   other_neurological |   chronic_pulmonary |   diabetes_uncomplicated |   diabetes_complicated |   hypothyroidism |   renal_failure |   liver_disease |   peptic_ulcer |   aids |   lymphoma |   metastatic_cancer |   solid_tumor |   rheumatoid_arthritis |   coagulopathy |   obesity |   weight_loss |   fluid_electrolyte |   blood_loss_anemia |   deficiency_anemias |   alcohol_abuse |   drug_abuse |   psychoses |   depression |
|---:|------------:|-------------:|-----------:|---------------------------:|----------------------:|-------------------:|------------------------:|----------------------:|---------------:|------------:|---------------------:|--------------------:|-------------------------:|-----------------------:|-----------------:|----------------:|----------------:|-------------

,hadm_id,subject_id,stay_num,congestive_heart_failure,cardiac_arrhythmias,valvular_disease,pulmonary_circulation,peripheral_vascular,hypertension,paralysis,...,coagulopathy,obesity,weight_loss,fluid_electrolyte,blood_loss_anemia,deficiency_anemias,alcohol_abuse,drug_abuse,psychoses,depression
8,20000147,14990224.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49,20001305,16003661.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
51,20001361,14577567.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
55,20001395,16679562.0,3.0,0.0,0.0,0.0,0.0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70,20001811,10926139.0,2.0,0.0,0.0,0.0,0.0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


time: 234 ms (started: 2023-07-26 14:39:45 +08:00)


In [39]:
elixhauser_nosummary_df.loc[elixhauser_nosummary_df['subject_id']==14990224.0]

,hadm_id,subject_id,stay_num,congestive_heart_failure,cardiac_arrhythmias,valvular_disease,pulmonary_circulation,peripheral_vascular,hypertension,paralysis,...,coagulopathy,obesity,weight_loss,fluid_electrolyte,blood_loss_anemia,deficiency_anemias,alcohol_abuse,drug_abuse,psychoses,depression
8,20000147,14990224.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
381524,28710206,14990224.0,2.0,0.0,0.0,0.0,0.0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


time: 31 ms (started: 2023-07-25 15:37:18 +08:00)


|    |   hadm_id |   subject_id |   stay_num |   congestive_heart_failure |   cardiac_arrhythmias |   valvular_disease |   pulmonary_circulation |   peripheral_vascular |   hypertension |   paralysis |   other_neurological |   chronic_pulmonary |   diabetes_uncomplicated |   diabetes_complicated |   hypothyroidism |   renal_failure |   liver_disease |   peptic_ulcer |   aids |   lymphoma |   metastatic_cancer |   solid_tumor |   rheumatoid_arthritis |   coagulopathy |   obesity |   weight_loss |   fluid_electrolyte |   blood_loss_anemia |   deficiency_anemias |   alcohol_abuse |   drug_abuse |   psychoses |   depression |
|---:|----------:|-------------:|-----------:|---------------------------:|----------------------:|-------------------:|------------------------:|----------------------:|---------------:|------------:|---------------------:|--------------------:|-------------------------:|-----------------------:|-----------------:|----------------:|----------------:|---------------:|-------:|-----------:|--------------------:|--------------:|-----------------------:|---------------:|----------:|--------------:|--------------------:|--------------------:|---------------------:|----------------:|-------------:|------------:|-------------:|
|  0 |    100001 |        58526 |          1 |                          0 |                     0 |                  0 |                       0 |                     0 |              1 |           0 |                    0 |                   0 |                        0 |                      1 |                0 |               1 |               0 |              0 |      0 |          0 |                   0 |             0 |                      0 |              0 |         0 |             0 |                   0 |                   0 |                    0 |               0 |            0 |           0 |            0 |
|  1 |    100003 |        54610 |          1 |                          0 |                     0 |                  0 |                       0 |                     0 |              1 |           0 |                    0 |                   0 |                        0 |                      0 |                0 |               0 |               1 |              0 |      0 |          0 |                   0 |             0 |                      0 |              0 |         0 |             0 |                   0 |                   0 |                    0 |               0 |            0 |           0 |            0 |
|  2 |    100006 |         9895 |          1 |                          0 |                     1 |                  0 |                       0 |                     0 |              0 |           0 |                    0 |                   0 |                        0 |                      0 |                0 |               0 |               0 |              0 |      0 |          1 |                   0 |             0 |                      0 |              0 |         0 |             0 |                   1 |                   0 |                    0 |               0 |            0 |           0 |            1 |
|  3 |    100007 |        23018 |          1 |                          0 |                     0 |                  0 |                       0 |                     0 |              1 |           0 |                    0 |                   0 |                        0 |                      0 |                0 |               0 |               0 |              0 |      0 |          0 |                   0 |             0 |                      0 |              0 |         0 |             0 |                   0 |                   0 |                    0 |               0 |            0 |           0 |            0 |
|  4 |    100009 |          533 |          1 |                          0 |                     1 |                  0 |                       0 |                     1 |              1 |           0 |                    0 |                   0 |                        1 |                      0 |                0 |               0 |               0 |              0 |      0 |          0 |                   0 |             0 |                      0 |              0 |         1 |             0 |                   0 |                   0 |                    0 |               0 |            0 |           0 |            0 |

### sum the cormorbidities but EXCLUDE CURRENT ROW FROM CUMMAX()

In [17]:
def elix(shift=False):
    elix_var=list(elixhauser_nosummary_df)[3:]
    elixhauser_nosummary_df2=elixhauser_nosummary_df.copy()

    elixhauser_nosummary_df2[elix_var]=(elixhauser_nosummary_df
                                            .sort_values('stay_num', ascending=True) #sorts values so stay_num is ascending
                                            .groupby('subject_id', as_index=False)[elix_var] #groups by subject id and filters only elixhauser variable columns
                                            .agg('cummax') #takes a cummulitive max for every row
                                       )

    if shift==True:
        #now shifting the values up by 1 so the cumulitive max doesn't consider the current values: (note: couldn't get this to work in the fxn above)
        elixhauser_nosummary_df2[elix_var]=(elixhauser_nosummary_df
                                                .sort_values('stay_num', ascending=True) #sorts values so stay_num is ascending
                                                .groupby('subject_id', as_index=False)[elix_var] #groups by subject id and filters only elixhauser variable columns
                                                .shift(fill_value=np.nan)[:-1]) #shifts the cummulitive max up by 1 so the first row is na. 

    #restricting to hadm in use
    elixhauser_nosummary_df3= elixhauser_nosummary_df2[elixhauser_nosummary_df2['hadm_id'].isin(hadm_id)]
    
    #adding icustay_id
    elixhauser_nosummary_df3=pd.merge(elixhauser_nosummary_df3, final_pt_df2[['hadm_id','stay_id']], how="left", left_on='hadm_id',right_on='hadm_id')
    
    return(elixhauser_nosummary_df3)

time: 0 ns (started: 2023-07-26 14:39:57 +08:00)


In [21]:
elixhauser_df=elix(shift=False)
cancer_elix=elixhauser_df[['subject_id','hadm_id','stay_id']].copy()
cancer_elix['value']=elixhauser_df.loc[:,['lymphoma',"solid_tumor","metastatic_cancer"]].max(axis=1)
#adding columns
cancer_elix['label']= 'cancer_elix'
cancer_elix['delta']= 0
cancer_elix['delta']= pd.to_timedelta(cancer_elix['delta'], unit='d')
cancer_elix['uom']= 'pos/neg category'

filter_subset_save(cancer_elix, savename="cancer_elix", save=save_boolean, return_df=False,time_filter_override=True) #filtering to ppl with sufficient vitals
#del(elixhauser_df,cancer_elix)

elixhauser_df=elix(shift=True)
elix_var=list(elixhauser_df)[3:-1]
sum_elix=elixhauser_df[['subject_id','hadm_id','stay_id']].copy()
sum_elix['value']=elixhauser_df.loc[:,elix_var].sum(axis=1)
#adding columns
sum_elix['label']= 'sum_elix'
sum_elix['delta']= 0
sum_elix['delta']= pd.to_timedelta(sum_elix['delta'], unit='d')
sum_elix['uom']= 'elixhauser_comorb_sum'

filter_subset_save(sum_elix, savename="sum_elix", save=save_boolean, return_df=False,time_filter_override=True) #filtering to ppl with sufficient vitals

time: 1.53 s (started: 2023-07-26 14:41:11 +08:00)


In [22]:
del(elixhauser_df,sum_elix)

time: 0 ns (started: 2023-07-26 14:57:42 +08:00)


In [47]:
cancer_elix

,subject_id,hadm_id,stay_id,value,label,delta,uom
0,14990224.0,20000147,30503572.0,0.0,cancer_elix,0 days,pos/neg category
1,16003661.0,20001305,36916968.0,0.0,cancer_elix,0 days,pos/neg category
2,14577567.0,20001361,33475095.0,0.0,cancer_elix,0 days,pos/neg category
3,11540283.0,20003491,33494445.0,0.0,cancer_elix,0 days,pos/neg category
4,18953418.0,20003543,31193392.0,0.0,cancer_elix,0 days,pos/neg category
...,...,...,...,...,...,...,...
34816,18164304.0,29997500,34008495.0,0.0,cancer_elix,0 days,pos/neg category
34817,19989305.0,29998115,36976872.0,0.0,cancer_elix,0 days,pos/neg category
34818,16968104.0,29998702,37074618.0,0.0,cancer_elix,0 days,pos/neg category
34819,15129243.0,29999098,33475199.0,0.0,cancer_elix,0 days,pos/neg category


time: 15 ms (started: 2023-07-25 15:48:40 +08:00)


# demographics

In [63]:
def demographics():
    """
    wrapping demographics code into a fxn. basically combines ethinicity, age, gender and race into one df. 
    mimiciv does not have religion
    """
    global final_pt_df2
    pt_info_sql = query_schema + """
    SELECT SUBJECT_ID, INSURANCE, LANGUAGE, MARITAL_STATUS, RACE
    from mimiciv_hosp.admissions
    ORDER BY subject_id DESC
    """

    pt_info_df=pd.read_sql_query(pt_info_sql,conn) #361711 patients with sterile culture -> 374643 with addn of bal and broncho... 7/16/18
    
    ethnicity_df=(pt_info_df.loc[
    pt_info_df.loc[:,"subject_id"].isin(
        final_pt_df2['subject_id'].tolist()),:]).drop_duplicates(['subject_id','race'])
    ethnicity_df= ethnicity_df[['subject_id','race']].sort_values('race', ascending=False).groupby('subject_id', as_index=False).first()
    
    #gender and age
    pt_info_sql = query_schema + """
    SELECT icu.stay_id,
        icu.subject_id,
        pd.anchor_age as yearsold,
        pd.gender
    FROM  mimiciv_icu.icustays icu
    LEFT JOIN mimiciv_hosp.patients pd
        ON pd.subject_id = icu.subject_id
    """
    
    #admissions
    pt_info_df=pd.read_sql_query(pt_info_sql,conn) #361711 patients with sterile culture -> 374643 with addn of bal and broncho... 7/16/18
    
    ## formatting yearsold so that above 89=90
    pt_info_df[pt_info_df['yearsold']>89]['yearsold']=90
    
    ## adding in icustay_id and t_0
    pt_info_df=pd.merge(pt_info_df, final_pt_df2[['stay_id','t_0']])
    
    #combining (age, gender) with ethnicity_df
    pt_info_df=(pt_info_df.loc[
        pt_info_df.loc[:,"stay_id"].isin(
            final_pt_df2['stay_id'].tolist()),:]).drop_duplicates(['stay_id','gender'])
    pt_info_df=(pt_info_df.loc[pt_info_df.loc[:,"stay_id"].isin(stay_id),:])
    pt_info_df= pd.merge(pt_info_df, ethnicity_df, left_on='subject_id', right_on='subject_id', how='left' )

    ## melting this combined df to get into long format
    age_df=pd.melt(pt_info_df, id_vars=['stay_id','subject_id','t_0'])
    age_df=age_df.rename(index=str, columns={'variable':'label'})

    age_df['delta']=pd.to_timedelta('0 days')
    age_df['uom']="N/A"
    age_df.loc[age_df.loc[:,'label']=='yearsold','uom']='yearsold'

    age_df= age_df.loc[age_df.loc[:,"stay_id"].isin(stay_id),:]

    ###using regular expressions to reduce the # of ethinicities
    age_df.loc[(age_df.loc[:,"label"]=='race') & (age_df.loc[:,"value"].str.contains(r'.*?BLACK')),'value']="black"
    age_df.loc[(age_df.loc[:,"label"]=='race') & (age_df.loc[:,"value"].str.contains(r'.*?HISPANIC|PORTUGUESE')),'value']="hispanic"
    age_df.loc[(age_df.loc[:,"label"]=='race') & (age_df.loc[:,"value"].str.contains(r'.*?WHITE')),'value']="white/nonhispanic"
    age_df.loc[(age_df.loc[:,"label"]=='race') & (age_df.loc[:,"value"].str.contains(r'.*?ASIAN')),'value']='asian'
    age_df.loc[(age_df.loc[:,"label"]=='race') & (age_df.loc[:,"value"].str.contains(r'(UNKNOWN|MULTI|UNABLE|DECLINE|OTHER)')),'value']='unknown/other'
    age_df.loc[(age_df.loc[:,"label"]=='race') & (age_df.loc[:,"value"].str.contains(r'[AZ]+')),'value']="unknown/other" #lumping all other low n values into other
    return(age_df)


time: 15 ms (started: 2023-07-25 15:59:36 +08:00)


In [68]:
age_df= demographics()

C:\Users\csfhe\AppData\Local\Temp\ipykernel_7228\857943213.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pt_info_df=pd.read_sql_query(pt_info_sql,conn) #361711 patients with sterile culture -> 374643 with addn of bal and broncho... 7/16/18
C:\Users\csfhe\AppData\Local\Temp\ipykernel_7228\857943213.py:32: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pt_info_df=pd.read_sql_query(pt_info_sql,conn) #361711 patients with sterile culture -> 374643 with addn of bal and broncho... 7/16/18
C:\Users\csfhe\AppData\Local\Temp\ipykernel_7228\857943213.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

time: 2.14 s (started: 2023-07-25 16:00:43 +08:00)


C:\Users\csfhe\AppData\Local\Temp\ipykernel_7228\857943213.py:62: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  age_df.loc[(age_df.loc[:,"label"]=='race') & (age_df.loc[:,"value"].str.contains(r'(UNKNOWN|MULTI|UNABLE|DECLINE|OTHER)')),'value']='unknown/other'


In [69]:
age_df

,stay_id,subject_id,t_0,label,value,delta,uom
0,37510196,10001884,2131-01-11 08:00:00,yearsold,68,0 days,yearsold
1,32358465,10002155,2131-03-10 01:00:00,yearsold,80,0 days,yearsold
2,33685454,10002155,2129-08-05 14:00:00,yearsold,80,0 days,yearsold
3,31090461,10002155,2130-09-25 11:00:00,yearsold,80,0 days,yearsold
4,36753294,10002495,2141-05-22 23:00:00,yearsold,81,0 days,yearsold
...,...,...,...,...,...,...,...
79408,36980198,19997367,2127-04-02 09:00:00,race,white/nonhispanic,0 days,N/A
79409,31417783,19998330,2178-10-22 18:00:00,race,black,0 days,N/A
79410,32641669,19998330,2178-10-02 13:00:00,race,black,0 days,N/A
79411,34403689,19998878,2132-10-01 00:00:00,race,white/nonhispanic,0 days,N/A


time: 15 ms (started: 2023-07-25 16:00:49 +08:00)


In [66]:
folder

'24_hr_window'

time: 0 ns (started: 2023-07-25 16:00:10 +08:00)


In [67]:
from parameters import save_boolean
filter_subset_save(age_df, savename="pt_info", save=save_boolean, return_df=False, time_filter_override=True)
del(age_df)

time: 1.11 s (started: 2023-07-25 16:00:19 +08:00)


## Weight

In [172]:
##reconnect to server if it dropsout
conn = psycopg2.connect(dbname=config.dbname, user=config.user, host=config.host, port=config.port,password=config.password)
cur=conn.cursor()
query_schema = 'SET search_path to ' + "mimiciv" + ';'


time: 94 ms (started: 2023-07-25 18:04:01 +08:00)


In [28]:
#prereq to weight and height
#mimic iv does not have this table
# echo_df= clinvar_fxn(
#     'echodata',
#     str(repository_path)+'/src/SQL/echodata.sql' 
# )

(15893, 18)
time: 35.5 s


In [72]:
# -- This query extracts weights for adult ICU patients on their first ICU day.
# -- It does *not* use any information after the first ICU day, as weight is
# -- sometimes used to monitor fluid balance.

weightfirstday_df= clinvar_fxn(
    'weightfirstday',
    str(repository_path)+'/src/SQL/weightfirstday.sql' 
)
weightfirstday_df['uom']='kg'

(27220, 4)
time: 2min (started: 2023-07-25 16:23:07 +08:00)


In [73]:
weightfirstday_df

,stay_id,weight,weight_admit,weight_daily,uom
2,30000484,68.50,68.5,NaN,kg
3,30000646,None,NaN,NaN,kg
4,30001148,65.70,65.7,69.9,kg
7,30001446,119.30,119.3,NaN,kg
9,30001555,None,NaN,NaN,kg
...,...,...,...,...,...
73171,39998622,130.00,NaN,130.0,kg
73173,39999172,70.00,70.0,NaN,kg
73174,39999230,59.50,59.5,59.5,kg
73176,39999301,107.70,107.7,NaN,kg


time: 15 ms (started: 2023-07-25 16:25:54 +08:00)


In [71]:
filter_subset_save(weightfirstday_df, savename="weightfirstday", save=save_boolean, return_df=False, time_filter_override=True)
#del(weightfirstday_df)

time: 125 ms (started: 2023-07-25 16:22:46 +08:00)


## Height

In [74]:
# -- This query extracts heights for adult ICU patients.
# -- It uses all information from the patient's first ICU day.
# -- This is done for consistency with other queries - it's not necessarily needed.
# -- Height is unlikely to change throughout a patient's stay.
heightfirstday_df= clinvar_fxn(
    'heightfirstday',
    str(repository_path)+'/src/SQL/heightfirstday.sql' 
)

heightfirstday_df['uom']='cm'

(27220, 2)
time: 1min (started: 2023-07-25 16:27:58 +08:00)


In [75]:
heightfirstday_df

,stay_id,height,uom
2,37510196,157.0,cm
4,32358465,NaN,cm
5,33685454,NaN,cm
6,31090461,NaN,cm
8,36753294,170.0,cm
...,...,...,...
73166,36980198,NaN,cm
73170,31417783,NaN,cm
73175,32641669,NaN,cm
73176,34403689,183.0,cm


time: 0 ns (started: 2023-07-25 16:28:59 +08:00)


In [76]:
filter_subset_save(heightfirstday_df, savename="heightfirstday", save=save_boolean, return_df=False, time_filter_override=True)
#del(heightfirstday_df)

time: 78 ms (started: 2023-07-25 16:28:59 +08:00)


In [33]:
del(echo_df)

time: 10.1 ms


# Labs

In [79]:
# -- This query pivots lab values for all patients, then filtered to those in my cohort.
# -- Have confirmed that the unit of measurement is always the same: null or the correct unit

labs_all_nosummary_df= clinvar_fxn(
    'labs_all_nosummary',
    str(repository_path)+'/src/SQL/labs_all_nosummary.sql'
)

(8336507, 6)
time: 2min 27s (started: 2023-07-25 16:32:28 +08:00)


In [80]:
#importing unit of mesurements:
def uom_sql_import(file_path):
    if isinstance(file_path, str)== True:
        f = open(Path(file_path), 'r')
    else:
        f = open(Path(str(file_path)), 'r')

    SQL = open(file_path,'r').read()
    SQL_df= pd.read_sql_query(SQL,conn)   
    return(SQL_df)

time: 0 ns (started: 2023-07-25 16:34:55 +08:00)


In [82]:
lab_uom= uom_sql_import(Path(str(repository_path)+'/src/SQL/labs_uom.sql'))
labs_all_nosummary_df = pd.merge(labs_all_nosummary_df, lab_uom, left_on='label', right_on='label')

labs_all_nosummary_df[labs_all_nosummary_df['label']=='LYMPHO%']#.value_counts() #4-15-19: what is this, is this exploring absolute lymphocyte %?
#labs_all_nosummary_df['label'].value_counts()

labs_all_nosummary_df[labs_all_nosummary_df['label']=='WBC']['valuenum'].describe()
#labs_all_nosummary_df[labs_all_nosummary_df['label']=='PLATELET']['valuenum'].describe()


C:\Users\csfhe\AppData\Local\Temp\ipykernel_7228\2035555998.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  SQL_df= pd.read_sql_query(SQL,conn)


count    496791.000000
mean         11.726333
std           9.462066
min           0.090000
25%           6.900000
50%          10.100000
75%          14.400000
max         471.700000
Name: valuenum, dtype: float64

time: 41.8 s (started: 2023-07-25 16:35:39 +08:00)


In [83]:
labs_all_nosummary_df

,subject_id,hadm_id,stay_id,charttime,label,valuenum,uom
0,10001884,26184834,37510196,2131-01-08 05:11:00,HEMATOCRIT,35.6,%
1,10001884,26184834,37510196,2131-01-09 04:45:00,HEMATOCRIT,36.9,%
2,10001884,26184834,37510196,2131-01-10 04:35:00,HEMATOCRIT,38.7,%
3,10001884,26184834,37510196,2131-01-10 20:45:00,HEMATOCRIT,39.7,%
4,10001884,26184834,37510196,2131-01-11 06:31:00,HEMATOCRIT,36.0,%
...,...,...,...,...,...,...,...
8298385,19999828,25744818,36075953,2149-01-11 05:37:00,PTT,26.7,sec
8298386,19999828,25744818,36075953,2149-01-12 06:04:00,PTT,28.9,sec
8298387,19999840,21033226,38978960,2164-09-13 01:51:00,PTT,29.3,sec
8298388,19999987,23865745,36195440,2145-11-03 01:35:00,PTT,31.8,sec


time: 15 ms (started: 2023-07-25 16:36:21 +08:00)


In [84]:
lab_uom

,uom,label
0,g/dL,ALBUMIN
1,mEq/L,ANION GAP
2,%,BANDS
3,mEq/L,BICARBONATE
4,mg/dL,BILIRUBIN
5,mg/dL,BUN
6,mEq/L,CHLORIDE
7,mg/dL,CREATININE
8,mg/dL,GLUCOSE
9,%,HEMATOCRIT


time: 16 ms (started: 2023-07-25 16:36:21 +08:00)


In [85]:
filter_subset_save(labs_all_nosummary_df, savename="labs_all_nosummary", save=save_boolean, return_df=False, time_filter_override=False)
del(labs_all_nosummary_df)
del(lab_uom)

time: 21.5 s (started: 2023-07-25 16:36:21 +08:00)


## Glasgow Coma Scale

In [113]:
#modified on 8/28/18 to have the days annotation.
##--8/28/18: added in epoch as days, in order to help determine btwn t_0 and 72 hour for pts.
gcsall_days_df= clinvar_fxn(
    'gcsall_days', 
    str(repository_path)+'/src/SQL/gcsall_days.sql' 
)

(27220, 9)
time: 1min 17s (started: 2023-07-25 17:07:45 +08:00)


In [114]:
gcsall_days_df= gcsall_days_df.loc[gcsall_days_df['day'].notnull()]

time: 16 ms (started: 2023-07-25 17:09:02 +08:00)


In [133]:
gcsall_days_df['day']*24

2         24
3        120
4         24
7         48
9         24
        ... 
73171     24
73173     24
73174    168
73176     24
73178     24
Name: day, Length: 27123, dtype: object

time: 16 ms (started: 2023-07-25 17:19:53 +08:00)


In [90]:
final_pt_df2

,stay_id,ssc_charttime,ssc_id,icu_admit,ab_id,ab_start,ab_end,ab_ssc_delta,t_0,t_end,...,t_end_consec,subject_id,dod,ab_course,org_list,spec_type_list,first_pos_else_neg_ssc,sc_result,final_bin,hadm_id
0,37510196,2131-01-10 16:36:00,635,2131-01-11 04:20:05,14800413,2131-01-11 08:00:00,2131-01-20 12:00:00,0 days 15:24:00,2131-01-11 08:00:00,2131-01-20 12:00:00,...,2131-01-20 12:00:00,10001884,2131-01-20,full,NaN,NaN,2131-01-10 16:36:00,negative,C_neg/A_full,26184834
1,33685454,2129-08-05 15:54:00,796,2129-08-04 12:45:00,14801473,2129-08-05 14:00:00,2129-08-10 13:00:00,-1 days +22:06:00,2129-08-05 14:00:00,2129-08-10 13:00:00,...,2129-08-16 09:00:00,10002155,2131-03-10,full,NaN,NaN,2129-08-05 15:54:00,negative,C_neg/A_full,23822395
2,31090461,2130-09-25 11:04:00,822,2130-09-24 00:50:00,14801575,2130-09-25 11:00:00,2130-09-26 11:00:00,-1 days +23:56:00,2130-09-25 11:00:00,2130-09-26 11:00:00,...,2130-09-26 11:00:00,10002155,2131-03-10,partial,NaN,NaN,2130-09-25 11:04:00,negative,C_neg/A_partial,28994087
3,32358465,2131-03-10 00:58:00,828,2131-03-09 21:33:00,14801420,2131-03-10 01:00:00,2131-03-10 14:00:00,0 days 00:02:00,2131-03-10 01:00:00,2131-03-10 14:00:00,...,2131-03-10 14:00:00,10002155,2131-03-10,full,NaN,NaN,2131-03-10 00:58:00,negative,C_neg/A_full,20345487
4,38875437,2156-04-19 16:28:00,893,2156-04-19 18:11:19,14802091,2156-04-20 12:00:00,2156-04-21 11:00:00,0 days 19:32:00,2156-04-20 12:00:00,2156-04-21 11:00:00,...,2156-04-21 17:00:00,10002428,NaN,partial,NaN,NaN,2156-04-19 16:28:00,negative,C_neg/A_partial,28662225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27214,37692584,2197-07-26 04:00:00,3228500,2197-07-26 03:31:06,14796703,2197-07-26 08:00:00,2197-07-28 18:00:00,0 days 04:00:00,2197-07-26 08:00:00,2197-07-28 18:00:00,...,2197-07-28 18:00:00,19999287,2197-09-02,partial,NaN,NaN,2197-07-26 04:00:00,negative,C_neg/A_partial,22997012
27215,35165301,2197-08-05 14:57:00,3228511,2197-08-04 00:02:00,14796608,2197-08-04 20:00:00,2197-08-12 11:00:00,-1 days +05:03:00,2197-08-04 20:00:00,2197-08-12 11:00:00,...,2197-08-12 11:00:00,19999287,2197-09-02,full,NaN,NaN,2197-08-05 14:57:00,negative,C_neg/A_full,20175828
27216,37364566,2162-08-15 09:33:00,3228521,2162-08-16 05:48:32,14796834,2162-08-16 08:00:00,2162-08-17 12:00:00,0 days 22:27:00,2162-08-16 08:00:00,2162-08-17 12:00:00,...,2162-08-17 12:00:00,19999297,2162-08-23,partial,STAPH AUREUS COAG +,BLOOD CULTURE,2162-08-15 09:33:00,positive,C_pos/A_partial,21439025
27217,36075953,2149-01-08 20:00:00,3228661,2149-01-08 18:12:00,14798102,2149-01-08 20:00:00,2149-01-14 08:00:00,0 days 00:00:00,2149-01-08 20:00:00,2149-01-14 08:00:00,...,2149-01-14 08:00:00,19999828,NaN,full,NaN,NaN,2149-01-08 20:00:00,negative,C_neg/A_full,25744818


time: 94 ms (started: 2023-07-25 16:42:40 +08:00)


In [140]:
gcsall_days_df['day']

2        1
3        5
4        1
7        2
9        1
        ..
73171    1
73173    1
73174    7
73176    1
73178    1
Name: day, Length: 27123, dtype: object

time: 0 ns (started: 2023-07-25 17:22:48 +08:00)


In [151]:
#adding in icu_admit time and filtereing time_var to time window.
def gcs_72(gcsall_days_df,final_pt_df2, time_var='t_0', lower_window=0, upper_window=3):
    """
    written a while back, aims to adding in icu_admit time and filtereing time_var to time window.
    will use this with time_filter_override=True in my filter_subset_save()
    """
    
    ##merging gcsall_days_df with final_pt df in order to append on icustay_id, the time var, and ICU_admit
    gcsall_days_df_merge= pd.merge(
    gcsall_days_df,
    final_pt_df2[['stay_id','icu_admit',time_var]],
    left_on='stay_id',
    right_on='stay_id')

    gcsall_days_df_merge['day'] = gcsall_days_df_merge['day']-1 #putting the epoch days so that 0 = the first day
    
    #approximating the charttime of the time associated with each gcs score
    gcsall_days_df_merge['approx_charttime']=pd.to_timedelta((gcsall_days_df_merge['day']).astype('int')*24, unit='h') + pd.to_datetime(gcsall_days_df_merge['icu_admit'])
    
    # day # + ICU_admission day.
    gcsall_days_df_merge['admit_plus_day']= (
        pd.to_datetime(gcsall_days_df_merge['icu_admit'])
        + pd.to_timedelta((gcsall_days_df_merge['day']).astype('int'), unit='D')
    )
    
    #difference between the admission+epoch day - time_var.
    gcsall_days_df_merge['delta']= (
        pd.to_datetime(gcsall_days_df_merge['admit_plus_day']) - pd.to_datetime(gcsall_days_df_merge[time_var])
    )
    
    #filtering day windows
    gcsall_days_df_merge_72= (
        gcsall_days_df_merge.loc[gcsall_days_df_merge.loc[:,'delta']>= pd.Timedelta(days=lower_window),:])
    gcsall_days_df_merge_72= (
        gcsall_days_df_merge_72.loc[gcsall_days_df_merge_72.loc[:,'delta']<= pd.Timedelta(days=upper_window),:])
    return(gcsall_days_df_merge_72)

time: 0 ns (started: 2023-07-25 17:27:03 +08:00)


In [126]:
np.arange(5)

array([0, 1, 2, 3, 4])

time: 0 ns (started: 2023-07-25 17:16:13 +08:00)


In [145]:
gcsall_days_df['day'].astype('int').to_numpy()

array([1, 5, 1, ..., 7, 1, 1])

time: 15 ms (started: 2023-07-25 17:24:37 +08:00)


In [148]:
pd.to_timedelta(gcsall_days_df['day'].astype('int') * 24, unit='h')

2       1 days
3       5 days
4       1 days
7       2 days
9       1 days
         ...  
73171   1 days
73173   1 days
73174   7 days
73176   1 days
73178   1 days
Name: day, Length: 27123, dtype: timedelta64[ns]

time: 15 ms (started: 2023-07-25 17:25:21 +08:00)


In [152]:
gcs72_df = gcs_72(gcsall_days_df,final_pt_df2, time_var=time_var, lower_window=lower_window,upper_window=upper_window )
gcs72_df['uom']='GCS_score' #adding in uom


time: 110 ms (started: 2023-07-25 17:27:07 +08:00)


In [153]:
gcs72_df

,subject_id,hadm_id,stay_id,day,mingcs,gcsmotor,gcsverbal,gcseyes,endotrachflag,icu_admit,t_0,approx_charttime,admit_plus_day,delta,uom
3,10656173,25778760,30001555,0,13.0,6.0,4.0,3.0,0.0,2177-09-27 11:23:13,2177-09-27 11:00:00,2177-09-27 11:23:13,2177-09-27 11:23:13,0 days 00:23:13,GCS_score
5,15978672,26652960,30002654,0,15.0,6.0,5.0,4.0,0.0,2154-10-17 23:44:59,2154-10-17 20:00:00,2154-10-17 23:44:59,2154-10-17 23:44:59,0 days 03:44:59,GCS_score
10,10369174,24697158,30004144,1,12.0,6.0,4.0,2.0,0.0,2126-04-04 13:20:25,2126-04-04 22:00:00,2126-04-05 13:20:25,2126-04-05 13:20:25,0 days 15:20:25,GCS_score
12,14289094,23247126,30005085,2,14.0,6.0,4.0,4.0,0.0,2136-01-26 10:37:03,2136-01-27 21:00:00,2136-01-28 10:37:03,2136-01-28 10:37:03,0 days 13:37:03,GCS_score
22,15993831,21791028,30009339,1,3.0,1.0,1.0,1.0,0.0,2145-05-09 23:35:00,2145-05-10 04:00:00,2145-05-10 23:35:00,2145-05-10 23:35:00,0 days 19:35:00,GCS_score
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26395,10128085,27136294,39990093,3,9.0,5.0,1.0,3.0,0.0,2145-08-06 15:10:17,2145-08-09 14:00:00,2145-08-09 15:10:17,2145-08-09 15:10:17,0 days 01:10:17,GCS_score
26397,16263223,22904714,39990860,1,13.0,6.0,4.0,3.0,0.0,2155-01-02 23:44:01,2155-01-03 13:00:00,2155-01-03 23:44:01,2155-01-03 23:44:01,0 days 10:44:01,GCS_score
26403,13534199,23907399,39995213,0,15.0,6.0,5.0,4.0,0.0,2127-12-11 18:03:43,2127-12-11 16:00:00,2127-12-11 18:03:43,2127-12-11 18:03:43,0 days 02:03:43,GCS_score
26407,19046950,24352151,39998622,0,3.0,1.0,1.0,1.0,0.0,2135-02-11 18:13:55,2135-02-11 18:00:00,2135-02-11 18:13:55,2135-02-11 18:13:55,0 days 00:13:55,GCS_score


time: 15 ms (started: 2023-07-25 17:27:09 +08:00)


In [154]:
filter_subset_save(gcs72_df, savename="gcs", save=save_boolean, return_df=False, time_filter_override=True)
del(gcs72_df)

time: 219 ms (started: 2023-07-25 17:27:13 +08:00)


## Renal replacement therapy

the sql code for this was not equipped to join all of the charttimes together. so i decided to do it in python below.
the rrt_all_df code above was only a 1 or 0 if patient had RRT during their entire icu stay. 

- step 1: run all sql codes
- 2: filter on only the t_0 to t_72 hour rows
- 3: filter on the 1223 patients who have a positive value
- 4: get the earliest incidence of rrt for each 1223 patients. 

In [161]:
def rrt_runmerge():
    """
    wrapping a lot of scripting into a function. grabs the 5 different rrt datas, filters them to timewindow, and merges them into 1 dataframe.
    """
    global date,folder, patient_df,lower_window, upper_window, time_var, time_var, time_col
    
    ###5 sql queries to grab raw data
    
    #mv_ce
    f = open(str(repository_path)+'/src/SQL/rtt_mv_ce.sql' , 'r')  
    rrtSQL_mv_ce = f.read()
    rrtSQL_mv_ce_sql = query_schema + rrtSQL_mv_ce.format(tuple(patients))
    rrtSQL_mv_ce_df=pd.read_sql_query(rrtSQL_mv_ce_sql,conn)    
    #cv
    f = open(str(repository_path)+'/src/SQL/rtt_cv.sql' , 'r') 
    rrtSQL_cv = f.read()
    rrtSQL_cv_sql = query_schema + rrtSQL_cv.format(tuple(patients))
    rrtSQL_cv_df=pd.read_sql_query(rrtSQL_cv_sql,conn)          
    #mv_ie
    f = open(str(repository_path)+'/src/SQL/rtt_mv_ie.sql' , 'r') 
    rrtSQL_mv_ie = f.read()
    rrtSQL_mv_ie_sql = query_schema + rrtSQL_mv_ie.format(tuple(patients))
    rrtSQL_mv_ie_df=pd.read_sql_query(rrtSQL_mv_ie_sql,conn)      
    rrtSQL_mv_ie_df['charttime']= rrtSQL_mv_ie_df['starttime']
    rrtSQL_mv_ie_df=rrtSQL_mv_ie_df.drop('starttime', axis=1)
    #mv_de
    f = open(str(repository_path)+'/src/SQL/rtt_mv_de.sql' , 'r') 
    rrtSQL_mv_de = f.read()
    rrtSQL_mv_de_sql = query_schema + rrtSQL_mv_de.format(tuple(patients))
    rrtSQL_mv_de_df=pd.read_sql_query(rrtSQL_mv_de_sql,conn)      
    #mv_pe
    f = open(str(repository_path)+'/src/SQL/rtt_mv_pe.sql' , 'r') 
    rrtSQL_mv_pe = f.read()
    rrtSQL_mv_pe_sql = query_schema + rrtSQL_mv_pe.format(tuple(patients))
    rrtSQL_mv_pe_df=pd.read_sql_query(rrtSQL_mv_pe_sql,conn)          
    rrtSQL_mv_pe_df['charttime']= rrtSQL_mv_pe_df['starttime']
    rrtSQL_mv_pe_df=rrtSQL_mv_pe_df.drop('starttime', axis=1)
    
    ### timewindow filtering

    def hour_72_window_rrt(df, final_pt_df2,timecol='charttime',time_var='t_0', lower_window=0, upper_window=3 ):
        ##modified to make more generalizable to easily accomidate PA cohort but default to my origional cohort.
        ##filters rrt to within timewindow  between timecol- time_var

        df= pd.merge(final_pt_df2[['stay_id',time_var]], df, left_on= 'stay_id', right_on = 'stay_id', how='left') #n=240317
        df['delta']= pd.to_datetime(df[timecol]) - pd.to_datetime(df[time_var])
        df_after_t0= df.loc[df.loc[:,'delta']>= pd.Timedelta(days=lower_window),:]
        df_after_t0= df_after_t0.loc[df_after_t0.loc[:,'delta']<= pd.Timedelta(days=upper_window),:] 
        #df_after_t0= df_after_t0.loc[df_after_t0.loc[:,'rrt']==1,:].groupby('stay_id')['charttime'].min()
        return(pd.DataFrame(df_after_t0))#.reset_index())
    
    rrtSQL_mv_ce_pt =hour_72_window_rrt(rrtSQL_mv_ce_df, patient_df, timecol=time_col,time_var=time_var, lower_window=lower_window,upper_window=upper_window)
    rrtSQL_cv_pt =hour_72_window_rrt(rrtSQL_cv_df, patient_df, timecol=time_col,time_var=time_var, lower_window=lower_window,upper_window=upper_window)
    rrtSQL_mv_ie_pt =hour_72_window_rrt(rrtSQL_mv_ie_df, patient_df, timecol=time_col,time_var=time_var, lower_window=lower_window,upper_window=upper_window)
    rrtSQL_mv_de_pt =hour_72_window_rrt(rrtSQL_mv_de_df, patient_df, timecol=time_col,time_var=time_var, lower_window=lower_window,upper_window=upper_window)
    rrtSQL_mv_pe_pt =hour_72_window_rrt(rrtSQL_mv_pe_df, patient_df, timecol=time_col,time_var=time_var, lower_window=lower_window,upper_window=upper_window)
    
    ### merging all 5 filtered rrt_df together
    
    def rrt_merging(rrtSQL_mv_ce_pt, rrtSQL_cv_pt, rrtSQL_mv_ie_pt, rrtSQL_mv_de_pt, rrtSQL_mv_pe_pt, timecol='charttime',time_var='t_0'):
        ###returns an aggregate y/n of if patient had positive rrt within timewindow. 

        rrt_merged_pt= pd.concat([rrtSQL_mv_ce_pt, rrtSQL_cv_pt, rrtSQL_mv_ie_pt, rrtSQL_mv_de_pt, rrtSQL_mv_pe_pt])

        #making a 1 if has positive rrt within timewindow:
        rrt_merged_pt= pd.DataFrame(rrt_merged_pt.loc[rrt_merged_pt.loc[:,'rrt']==1,:].groupby('stay_id')[timecol].min().reset_index())
        rrt_merged_pt['rrt']=1

        rrt_merged_allpt_df= pd.merge(final_pt_df2[['stay_id',time_var]], rrt_merged_pt, left_on= 'stay_id', right_on = 'stay_id', how='left') #n=240317
        rrt_merged_allpt_df=rrt_merged_allpt_df.rename(index=str, columns={timecol:"first_charttime"})

        rrt_merged_allpt_df['uom']='category' #adding a uom category
        rrt_merged_allpt_df.loc[rrt_merged_allpt_df.loc[:,'rrt'].isnull(),'rrt']='0'

        return(rrt_merged_allpt_df)


    rrt_merged_allpt_df= rrt_merging(rrtSQL_mv_ce_pt, rrtSQL_cv_pt, rrtSQL_mv_ie_pt, rrtSQL_mv_de_pt, rrtSQL_mv_pe_pt, timecol=time_col,time_var=time_var)
    return(rrt_merged_allpt_df)

time: 16 ms (started: 2023-07-25 17:42:06 +08:00)


In [163]:
rrt_merged_allpt_df= rrt_runmerge()
filter_subset_save(rrt_merged_allpt_df, savename="rrt_merged", save=save_boolean, return_df=False, time_filter_override=True)
#del(rrt_merged_allpt_df)

C:\Users\csfhe\AppData\Local\Temp\ipykernel_7228\1325457263.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  rrtSQL_mv_ce_df=pd.read_sql_query(rrtSQL_mv_ce_sql,conn)
C:\Users\csfhe\AppData\Local\Temp\ipykernel_7228\1325457263.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  rrtSQL_cv_df=pd.read_sql_query(rrtSQL_cv_sql,conn)
C:\Users\csfhe\AppData\Local\Temp\ipykernel_7228\1325457263.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  rrtSQL_mv_ie_df=pd.read_sql_query(rrtSQL_mv_ie_sql,conn)
C:\Users\csfhe\AppData\Lo

time: 2min 12s (started: 2023-07-25 17:53:29 +08:00)


In [164]:
rrt_merged_allpt_df

,stay_id,t_0,first_charttime,rrt,uom
0,37510196,2131-01-11 08:00:00,NaT,0,category
1,33685454,2129-08-05 14:00:00,NaT,0,category
2,31090461,2130-09-25 11:00:00,NaT,0,category
3,32358465,2131-03-10 01:00:00,NaT,0,category
4,38875437,2156-04-20 12:00:00,NaT,0,category
...,...,...,...,...,...
26466,37692584,2197-07-26 08:00:00,NaT,0,category
26467,35165301,2197-08-04 20:00:00,NaT,0,category
26468,37364566,2162-08-16 08:00:00,NaT,0,category
26469,36075953,2149-01-08 20:00:00,NaT,0,category


time: 15 ms (started: 2023-07-25 17:56:06 +08:00)


# UTI related variables

In [17]:
uti_all_df= clinvar_fxn(
    'uti_all',
    str(repository_path)+'/src/SQL/uti_all.sql'
    #str(repository_path)+'/src/SQL/gcsall_days.sql' 
)

MemoryError: Unable to allocate 2.85 GiB for an array with shape (5, 76589793) and data type object

time: 21min 17s (started: 2023-07-26 12:13:26 +08:00)


In [ ]:
filter_subset_save(gcsall_days_df, savename="uti_all", save=save_boolean, return_df=False, time_filter_override=False)
del(uti_all_df)
#(5, 76589793)

## Blood Gas Test

In [5]:
def PaO2(bg_all_nosummary_df):
    """
    overview: replaces the PO2 label with PaO2 on all instances (defined as sharing icustay_id and charttime being equal)
        where the specimen label == 'ART'
        
    input: bloodgas dataframe with values annotated. 
    output: bloodgas dataframe with values annotated where PO2 label is replaced with PaO2 according to above criteria
    """
    
    #making a unique varaible to search for and mark all rows where time and icustay_id has an art flag
    bg_all_nosummary_df['unique_var']= bg_all_nosummary_df['stay_id'].map(str) +  bg_all_nosummary_df['charttime'].map(str)

    #making subset dataframe for label == SPECIMEN 
    bg_all_nosummary_specimen= bg_all_nosummary_df.loc[bg_all_nosummary_df.loc[:,'label']=='SPECIMEN',:]
    
    #all ART related rows: unique_var for all rows where label== SPECIMEN
    bg_all_nosummary_ART = bg_all_nosummary_specimen[bg_all_nosummary_specimen['value']=='ART'] 
    bg_all_nosummary_ART_list= list(bg_all_nosummary_ART['unique_var'].unique())

    #two criteria needed to change the PO2 to PaO2 label.
    criteria1=(bg_all_nosummary_df['label'] == 'PO2')
    criteria2=(bg_all_nosummary_df['unique_var'].isin(bg_all_nosummary_ART_list))
    
    #making changes
    bg_all_nosummary_df.loc[(criteria2 & criteria1),'label']= 'PaO2'
    
    return(bg_all_nosummary_df)

time: 16 ms (started: 2023-07-25 19:47:20 +08:00)


In [10]:
bg_all_nosummary_df= clinvar_fxn(
    'bg_all_nosummary',
    str(repository_path)+'/src/SQL/bg_all_nosummary.sql' 
    ) 
bg_all_nosummary_df = PaO2(bg_all_nosummary_df)


(2832044, 8)
time: 1min 48s (started: 2023-07-25 19:48:06 +08:00)


In [21]:
filter_subset_save(bg_all_nosummary_df, savename="bg_all_nosummary", save=save_boolean, return_df=False, time_filter_override=False)
del(bg_all_nosummary_df)

time: 18.9 s (started: 2023-07-25 20:00:46 +08:00)


## Vaso_active therapies

In [23]:
weightdurations_df= clinvar_fxn(
    'weightdurations',
    str(repository_path)+'/src/SQL/weightdurations.sql'  ##added to vasoactive_meds due to dependency of SQL code
)

(158362, 4)
time: 1min 27s (started: 2023-07-25 20:03:15 +08:00)


In [16]:
epi_dose_df= clinvar_fxn(
    'epinephrine_dose',
    str(repository_path)+'/src/SQL/epinephrine_dose.sql' 
)

(5470, 8)
time: 2.62 s (started: 2023-07-26 08:59:08 +08:00)


In [17]:
norepi_dose_df= clinvar_fxn(
    'norepinephrine_dose',
    str(repository_path)+'/src/SQL/norepinephrine_dose.sql' 
)

(46729, 8)
time: 5.34 s (started: 2023-07-26 09:00:10 +08:00)


In [20]:
dopamine_dose_df= clinvar_fxn(
    'dopamine_dose',
    str(repository_path)+'/src/SQL/dopamine_dose.sql' 
)

(3116, 8)
time: 500 ms (started: 2023-07-26 09:03:44 +08:00)


In [22]:
dobutamine_dose_df= clinvar_fxn(
    'dobutamine_dose',
    str(repository_path)+'/src/SQL/dobutamine_dose.sql' 
)

(3363, 8)
time: 469 ms (started: 2023-07-26 09:08:58 +08:00)


In [23]:
vasopressin_dose_df= clinvar_fxn(
    'vasopressin_dose',
    str(repository_path)+'/src/SQL/vasopressin_dose.sql' 
)
#removing units/hour, as these are not appropriate
vasopressin_dose_df= vasopressin_dose_df.loc[~vasopressin_dose_df.loc[:,'rate_uom'].isin(['Uhr','units/hour']),:]

(11591, 8)
time: 672 ms (started: 2023-07-26 09:11:59 +08:00)


In [24]:
phenylephrine_dose_df= clinvar_fxn(
    'phenylephrine_dose',
    str(repository_path)+'/src/SQL/phenylephrine_dose.sql' 
)

(23984, 8)
time: 2.08 s (started: 2023-07-26 09:13:03 +08:00)


In [25]:
#adding an identification label column and merging them into 1 df. 
epi_dose_df['label']='epinephrine'
norepi_dose_df['label']='norepinephrine'
dopamine_dose_df['label']='dopamine'
dobutamine_dose_df['label']='dobutamine'
vasopressin_dose_df['label']='vasopressin'
phenylephrine_dose_df['label']='phenylephrine'
vaso_dose_df = pd.concat([epi_dose_df, norepi_dose_df, dopamine_dose_df, dobutamine_dose_df, vasopressin_dose_df,phenylephrine_dose_df ])

#rename starttime to charttime
vaso_dose_df.rename(index=str, columns={'starttime':"charttime"}, inplace=True)

time: 47 ms (started: 2023-07-26 09:13:18 +08:00)


In [26]:
filter_subset_save(vaso_dose_df, savename="vaso_dose", save=save_boolean, return_df=False, time_filter_override=False)
vaso_dose_df

,stay_id,linkorderid,vaso_rate,vaso_amount,charttime,endtime,amount_uom,rate_uom,label
0,30009339,9395161,0.010000,0.048000,2145-05-10 07:00:00,2145-05-10 08:00:00,mg,mcg/kg/min,epinephrine
1,30009339,1700260,0.100000,0.960800,2145-05-10 10:00:00,2145-05-10 14:17:00,mg,mcg/kg/min,epinephrine
2,30009339,2346526,0.040085,1.993456,2145-05-14 09:10:00,2145-05-14 21:31:00,mg,mcg/kg/min,epinephrine
3,30009339,2182038,0.040091,1.906808,2145-05-14 21:31:00,2145-05-15 08:53:00,mg,mcg/kg/min,epinephrine
4,30009339,9875968,0.030096,0.488944,2145-05-15 08:53:00,2145-05-15 14:05:00,mg,mcg/kg/min,epinephrine
...,...,...,...,...,...,...,...,...,...
42234,39994129,9195559,0.300015,0.399020,2136-08-30 21:55:00,2136-08-30 22:14:00,mg,mcg/kg/min,phenylephrine
42269,39996783,4427252,5.005536,37.084658,2126-06-29 06:19:00,2126-06-29 09:16:00,mg,mcg/kg/min,phenylephrine
42270,39999552,5936118,0.200007,0.670824,2186-07-17 16:46:00,2186-07-17 17:38:00,mg,mcg/kg/min,phenylephrine
42271,39999552,779141,0.300075,1.064516,2186-07-17 19:30:00,2186-07-17 20:25:00,mg,mcg/kg/min,phenylephrine


time: 688 ms (started: 2023-07-26 09:13:48 +08:00)


In [32]:
del(vaso_dose_df)
del(epi_dose_df, norepi_dose_df, dopamine_dose_df, dobutamine_dose_df, vasopressin_dose_df,phenylephrine_dose_df)

time: 234 ms (started: 2023-07-26 11:35:36 +08:00)


# ventilator settings and categorization

In [33]:
ventsettings_df= clinvar_fxn(
    'ventsettings',
    str(repository_path)+'/src/SQL/ventsettings.sql' 
)
#going from wide format to long:
#pd.melt(ventsettings_df, id_vars=['icustay_id','charttime'])

(995200, 6)
time: 1min 52s (started: 2023-07-25 20:21:40 +08:00)


In [36]:
ventsettings_df.head()

,stay_id,charttime,mechvent,oxygentherapy,extubated,selfextubated
35,30000484,2136-01-14 18:46:00,0,1,0,0
36,30000484,2136-01-14 18:48:00,0,1,0,0
37,30000484,2136-01-14 22:00:00,0,1,0,0
38,30000484,2136-01-15 01:00:00,0,1,0,0
39,30000484,2136-01-15 02:00:00,0,1,0,0


time: 16 ms (started: 2023-07-25 20:24:41 +08:00)


In [38]:
def vent_data(vent_df,time_var='t_0', lower_window=0, upper_window=3 ):
    df= pd.merge(vent_df,
                final_pt_df2[['stay_id',time_var]],
                left_on='stay_id',
                right_on= 'stay_id',
                how='left')
    df['delta']= pd.to_datetime(df['charttime']) - pd.to_datetime(df[time_var])
    df_timewindow= df.loc[df.loc[:,'delta']>= pd.Timedelta(days=lower_window),:]
    df_timewindow= df_timewindow.loc[df_timewindow.loc[:,'delta']<= pd.Timedelta(days=upper_window),:] 
    df_timewindow['day']= df_timewindow['delta'].apply(lambda x: pd.to_timedelta(x,unit='d').days) #day #
      
    return(df_timewindow)

#df_timewindow =vent_data(ventsettings_df,time_var='first_pos_else_neg_ssc', lower_window=-1, upper_window=1 )

def vent_day_categorizer(vent_df,time_var='t_0', lower_window=0, upper_window=3 ):
    
    df_timewindow =vent_data(vent_df,time_var=time_var, lower_window=lower_window, upper_window=upper_window)
    
    df_timewindow_perday=df_timewindow.groupby(['stay_id','day'], as_index=False)[['mechvent','oxygentherapy']].agg({'mechvent':'max', 'oxygentherapy':'max'})

    conditions= [
        (df_timewindow_perday['mechvent']==1),
        ((df_timewindow_perday['oxygentherapy']==1) & (df_timewindow_perday['mechvent']==0)),
        (df_timewindow_perday['oxygentherapy']==0 & (df_timewindow_perday['mechvent']==0))]

    choices=['Mech', 'Oxygen', 'None']
    #
    df_timewindow_perday['value']= np.select(conditions, choices, default='error:no_value_filled')

    df_timewindow_perday=df_timewindow_perday.reset_index()
    df_timewindow_perday['uom']= 'mech/O2/none category'
    df_timewindow_perday= df_timewindow_perday.drop(['mechvent','oxygentherapy','index'], axis=1)
    df_timewindow_perday=pd.merge(df_timewindow_perday, final_pt_df2[['stay_id',time_var]] )
    return(df_timewindow_perday)


time: 0 ns (started: 2023-07-25 20:25:18 +08:00)


In [39]:
#ventcategory_df = vent_categorization(final_pt_df2, ventsettings_df, time_var='first_pos_else_neg_ssc' )
ventcategory_df= vent_day_categorizer(ventsettings_df,time_var=time_var, lower_window=lower_window, upper_window=upper_window)
#ventcount_df = vent_count(final_pt_df2,ventsettings_df, time_var='first_pos_else_neg_ssc')

time: 3.05 s (started: 2023-07-25 20:25:21 +08:00)


In [40]:
filter_subset_save(ventcategory_df, savename="ventcategory", save=save_boolean, return_df=False, time_filter_override=True)
#del(ventcategory_df)

time: 125 ms (started: 2023-07-25 20:25:24 +08:00)


In [41]:
ventcategory_df.head()

,stay_id,day,value,uom,t_0
0,30000484,0,Oxygen,mech/O2/none category,2136-01-14 21:00:00
1,30000484,1,None,mech/O2/none category,2136-01-14 21:00:00
2,30000646,0,Oxygen,mech/O2/none category,2194-04-29 07:00:00
3,30001446,0,Oxygen,mech/O2/none category,2186-04-12 01:00:00
4,30001555,0,Oxygen,mech/O2/none category,2177-09-27 11:00:00


time: 0 ns (started: 2023-07-25 20:25:25 +08:00)


# daily SOFA score

### running yiheng's sql codes to capture daily sofa_score. ideally i coulda just used my data above but she had this written already so i'll use this.

link to her github: https://github.com/yihengpan/fluid_management/tree/master/sofa

Here is how we generate the sofa table:


sofa <- scorecalc <- scorecomp <- vaso_cv    <-icu_18
                                             
                                             <-wt      <-icu_18
                                             <-echo2
                                          
                               <- vaso_mv    <-icu_18
                               <- pafi2      <-bloodgas_pan_aterial <- bloodgas_pan  <-icu_18
                                             <-ventelurations
                               <- vitals_pan <-icu_18
                               <- labs_pan   <-icu_18
                               <- uo_pan     <-icu_18
                               <- gcs_pan    <-icu_18



In [ ]:
sofa_path= str(repository_path)+'/src/SQL/sofa'
var='wt'
wt_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)

var='echo2'
echo2_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)

var='bloodgas_pan'
bloodgas_pan_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)

var='bloodgas_pan_arterial'
bloodgas_pan_art_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)

In [11]:
#vaso_mv, vaso_cv,pafi2, vitals_pan, labs_pan, uo_pan, gcs_pan
sofa_path= str(repository_path)+'/src/SQL/sofa'
var='vaso_mv'
vaso_mv_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)

var='vaso_mv'
vaso_mv_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)

var='vaso_cv'
vaso_cv_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)

var='pafi1'
pafi1_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)

var='pafi2'
pafi2_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)


var='labs_pan'
labs_pan_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)

var='uo_pan'
uo_pan_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)

var='gcs_pan'
gcs_pan_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)

###

var='scorecomp'
scorecomp_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)

var='scorecalc'
scorecalc_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)

var='sofa_pan'
sofa_pan_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)

(8209, 6)
(8209, 6)
(11257, 6)
(241035, 5)
(76566, 4)
(116400, 42)
(111927, 5)
(19692, 9)
(133916, 14)
(133916, 20)
(125866, 23)
time: 31min 20s


In [62]:
#deleting these to clear up memory 

del(vaso_cv_df, vaso_mv_df, labs_pan_df, gcs_pan_df, scorecalc_df, scorecomp_df, uo_pan_df, pafi1_df, pafi2_df, bloodgas_pan_art_df, echo2_df, wt_df)

time: 47.6 ms


In [12]:
print(
sofa_pan_df['icustay_id'].nunique(),
final_pt_df2['icustay_id'].nunique() 
)

19884 19633
time: 5.19 ms


In [13]:
#adding in t_0 & icuadmit date
def sofa_day_window_filter(sofa_pan_df, time_var= 't_0', lower_window= 0, upper_window=3): #'t_0'):
    import datetime
    '''
    #Yihangpan wrote a sql script and materialized view "sofa_pan" which gives the sofa score for each day in icu for each patient. 
    #since the sofa_pan has days after admission but not chartdates, I need to use day # to find the associated t_0 to t_0+72 for each patient. 
    # the challenge was that I had to relate day# in sofa_pan to my t_0 date. the day # was based on the days after icu admission, where day1 = the first day (day=0)= 0 to 24 hours post admission.
    #To do this, I added day# (where day 0 is the first day) to icu admission date. 
    #I then filtered on only the rows where this icuadmin + day# was between t_0 and t_0 + 72 hours. 

    #since t_0 has only day resolution, and for that I ignored hours and only took the date (rounded down all hours/minutes/seconds). this is similar to how i made the t_0 date. 
    #the problem this creates is that it widens the potential time window, so it theoretically can contain up to 95.99 hours, since hours on day 1 were collapsed to 0. 

    
    input: 
        sofa_pan_df: daily sofa scores captured from sofa_pan_sql.
        optional:
            time_var: the time variable we want to base the window off of
            window_bottom= 0, time_var- window_bottom (days + time_var) = first daily sofa score to capture
            window_top= 0, time_var- window_top (days + time_var) = last daily sofa score to capture 
    
    output: sofa_pan_sql annotated with days and filtered to time window set by window_bottom and window_top. 
    '''
    #reducing to minimum vital patients
    sofa_pan_df=sofa_pan_df.loc[sofa_pan_df.loc[:,"icustay_id"].isin(icustay_id_vitals),:].copy()

    ##merging sofa_pan with final_pt df in order to append on icustay_id, the time var, and ICU_admit
    
    sofa_df_merged= pd.merge(sofa_pan_df,
                             final_pt_df2[['icustay_id',time_var,'icu_admit']],
                             left_on= 'icustay_id',
                             right_on = 'icustay_id',
                             how='left') #n=240317

    #sofa_df_merged['admit_t0_rounded'] = pd.to_datetime(sofa_df_merged['ICU_admit']).dt.round('1440min')
    
    sofa_df_merged['day'] = sofa_df_merged['day']-1 #putting the epoch days so that 0 = the first day
    
    #approximating the charttime of the time associated with each sofa score. adding on days to icuadmit. 
    sofa_df_merged['approx_charttime']=pd.to_timedelta((sofa_df_merged['day'])*24, unit='h') + pd.to_datetime(sofa_df_merged['icu_admit'])

    #rounding down the charttime to the day, so hours and minutes are ignored (just like t_0)
    sofa_df_merged['floor_charttime'] = sofa_df_merged['approx_charttime'].apply(lambda dt: datetime.datetime(dt.year, dt.month, dt.day, 24*(dt.hour//24))) 
    
    sofa_df_merged['floor_time_var'] = pd.to_datetime(sofa_df_merged[time_var]).apply(lambda dt: datetime.datetime(dt.year, dt.month, dt.day, 24*(dt.hour//24))) 
    
    sofa_df_window= sofa_df_merged.loc[
        (sofa_df_merged['floor_charttime'].between(
            (pd.to_datetime(sofa_df_merged['floor_time_var'])+ pd.to_timedelta(lower_window, unit='d')),
            (pd.to_datetime(sofa_df_merged['floor_time_var'])+ pd.to_timedelta(upper_window, unit='d')+ pd.to_timedelta(1, unit='h')) #added 1hr timebuffer incase between is set as less than greater than
        )),:]
    
    return(sofa_df_window)
    #return(sofa_df_window.drop(['floor_time_var','floor_charttime'], axis=1))


time: 3.28 ms


In [19]:
sofa_df_window= sofa_day_window_filter(sofa_pan_df, time_var= time_var, lower_window= lower_window, upper_window=upper_window)
sofa_df_window.sort_values(['icustay_id','day','sofa','rate_dopamine','rate_epinephrine','rate_norepinephrine','rate_dobutamine'],
                          ascending=[True, True, False, False, False, False, False], inplace=True
                          )
sofa_df_window.drop_duplicates(['icustay_id','day','sofa'], inplace=True)
print(len(sofa_df_window))

filter_subset_save(sofa_df_window, savename="sofa", save=save_boolean, return_df=False, time_filter_override=True)


106097
/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-opensource/data/raw/30_hr_window/
time: 3.76 s


In [ ]:
#-- running the 30 day version for ab free day outcome calc; should be much more than 43650 

# end of extraction